## Insurance Forecast



**Proposta:** Utilizar algoritmos de aprendizado supervisionado para prever os custos do seguro de saúde a partir dos atributos dos beneficiários.

**Link do Kaggle: **https://www.kaggle.com/datasets/teertha/ushealthinsurancedataset

### Exploração de dados

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import scipy.stats as stats
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_absolute_error, mean_squared_error
from sklearn.neural_network import MLPRegressor
import statsmodels
from statsmodels.stats.diagnostic import lilliefors
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
!pip install catboost
from catboost.core import CatBoostRegressor

In [ ]:
dados = pd.read_csv('/content/drive/MyDrive/CursoML/Insurance/insurance.csv',sep=',', encoding='iso-8859-1')

In [ ]:
dados.shape

In [ ]:
dados.head()

In [ ]:
dados['age'].plot(kind='hist', bins=20, title='age')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dados.groupby('sex').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dados['bmi'].plot(kind='hist', bins=20, title='bmi')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dados['children'].plot(kind='hist', bins=20, title='children')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dados.groupby('smoker').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dados.groupby('region').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dados['charges'].plot(kind='hist', bins=20, title='charges')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dados.dtypes

In [ ]:
dados.isnull().sum()

In [ ]:
plt.scatter(dados.age, dados.bmi)
plt.title('Correlação Linear')
plt.xlabel('Idade')
plt.ylabel('IMC')
plt.grid(False)

In [ ]:
sns.pairplot(dados);

In [ ]:
stats.probplot(dados["age"], dist="norm", plot=plt)
plt.title("Normal Q-Q plot")
plt.show()

In [ ]:
hist = px.histogram (dados, x = "age", nbins=60)
hist.update_layout(width=800,height=500,title_text='Descrição do atributo')
hist.show()

In [ ]:
# Uníco que se assemelha a distribuição normal
hist = px.histogram (dados, x = "bmi", nbins=60)
hist.update_layout(width=800,height=500,title_text='Descrição do atributo')
hist.show()

In [ ]:
hist = px.histogram (dados, x = "charges", nbins=60)
hist.update_layout(width=800,height=500,title_text='Descrição do atributo')
hist.show()

In [ ]:
hist = px.histogram (dados, x = "children", nbins=60)
hist.update_layout(width=800,height=500,title_text='Descrição do atributo')
hist.show()

In [ ]:
estatistica, p = stats.shapiro(dados.bmi)
print('Estatística do teste: {}'.format(estatistica))
print('p-valor: {}'.format(p))

In [ ]:
estatistica, p = statsmodels.stats.diagnostic.lilliefors(dados.bmi, dist = 'norm')
print('Estatística de teste: {}'.format(estatistica))
print('p-valor: {}'.format(p))

Nenhuma das variáveis tem distribuição normal.

### Correlação linear

In [ ]:
coef, p = stats.spearmanr(dados.age, dados.children)
print('Coeficiente de correlação: {}'.format(coef))
print('p-valor: {}'.format(p))

In [ ]:
coef, p = stats.spearmanr(dados.sex, dados.smoker)
print('Coeficiente de correlação: {}'.format(coef))
print('p-valor: {}'.format(p))

Age e Children, Sex e Smoker parecem ter correlação.

### Tratamento dos dados

In [ ]:
previsores = dados.iloc[:, 0:6].values

In [ ]:
previsores2 = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [1,4,5])], remainder='passthrough').fit_transform(previsores)


In [ ]:
previsores_esc = StandardScaler().fit_transform(previsores2)

In [ ]:
alvo = dados.iloc[:,6]

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(previsores_esc, alvo, test_size=0.2, random_state=42)

### Regressão Linear Múltipla

In [ ]:
reg_linear1 = LinearRegression()
reg_linear1.fit(x_treino, y_treino)

In [ ]:
reg_linear1.intercept_

In [ ]:
reg_linear1.coef_

In [ ]:
reg_linear1.score(x_treino, y_treino)

In [ ]:
reg_linear1.score(x_teste, y_teste)

In [ ]:
previsoes_treino = reg_linear1.predict(x_treino)
previsoes_treino

In [ ]:
previsoes_teste = reg_linear1.predict(x_teste)
previsoes_teste

In [ ]:
abs(y_teste - previsoes_teste).mean()

In [ ]:
mean_absolute_error(y_teste, previsoes_teste)

In [ ]:
mean_squared_error(y_teste, previsoes_teste)

In [ ]:
np.sqrt(mean_squared_error(y_teste, previsoes_teste))

In [ ]:
kfold = KFold(n_splits = 15, shuffle=True, random_state = 5)

In [ ]:
modelo = LinearRegression()
resultado = cross_val_score(modelo, previsores_esc, alvo, cv = kfold)
resultado

In [ ]:
print("Coef. determinação Médio: %.2f%%" % (resultado.mean() * 100.0))

### Regressão por Vetores de Suporte

In [ ]:
SVR = SVR(kernel='rbf')
SVR.fit(x_treino,y_treino)

In [ ]:
SVR.score(x_treino, y_treino)

In [ ]:
SVR.score(x_teste, y_teste)

In [ ]:
x_scaler = StandardScaler()
x_treino_scaler = x_scaler.fit_transform(x_treino)

In [ ]:
y_scaler = StandardScaler()
y_treino_scaler = y_scaler.fit_transform(y_treino.values.reshape(-1,1))

In [ ]:
x_teste_scaler = x_scaler.transform(x_teste)
x_teste_scaler

In [ ]:
y_teste_scaler = y_scaler.transform(y_teste.values.reshape(-1,1))
y_teste_scaler

In [ ]:
from sklearn.svm import SVR
SVR2 = SVR(kernel='rbf')
SVR2.fit(x_treino_scaler, y_treino_scaler.ravel())

In [ ]:
SVR2.score(x_treino_scaler, y_treino_scaler)

In [ ]:
SVR2.score(x_teste_scaler, y_teste_scaler)

In [ ]:
y_teste_inverse = y_scaler.inverse_transform(y_teste_scaler)
previsoes_inverse = y_scaler.inverse_transform(previsoes_teste.reshape(-1, 1))

In [ ]:
mean_absolute_error(y_teste_inverse, previsoes_inverse)

In [ ]:
mean_squared_error(y_teste_inverse, previsoes_inverse)

In [ ]:
np.sqrt(mean_squared_error(y_teste_inverse, previsoes_inverse))

In [ ]:
y = StandardScaler()
alvo_esc = y.fit_transform(alvo.values.reshape(-1,1))

In [ ]:
kfold = KFold(n_splits = 15, shuffle=True, random_state = 5)

In [ ]:
from sklearn.svm import SVR
modelo = SVR(kernel='rbf')
resultado = cross_val_score(modelo, previsores_esc, alvo_esc.ravel(), cv = kfold)
resultado

In [ ]:
print("Coeficiente de Determinação médio: %.2f%%" % (resultado.mean() * 100.0))

### Regressão por Árvore de Decisão

In [ ]:
arvore = DecisionTreeRegressor(max_depth=4, random_state=10)
arvore.fit(x_treino, y_treino)

In [ ]:
arvore.score(x_treino, y_treino)

In [ ]:
arvore.score(x_teste, y_teste)

In [ ]:
previsoes_teste = arvore.predict(x_teste)
previsoes_teste

In [ ]:
mean_absolute_error(y_teste, previsoes_teste)

In [ ]:
mean_squared_error(y_teste, previsoes_teste)

In [ ]:
np.sqrt(mean_squared_error(y_teste, previsoes_teste))

In [ ]:
kfold = KFold(n_splits = 15, shuffle=True, random_state = 5)

In [ ]:
modelo = DecisionTreeRegressor(max_depth=4, random_state=10)
resultado = cross_val_score(modelo, previsores_esc, alvo, cv = kfold)
resultado

In [ ]:
print("Coeficiente de Determinação Médio: %.2f%%" % (resultado.mean() * 100.0))

### Regressão com Random Forest

In [ ]:
random = RandomForestRegressor(n_estimators=100, criterion='squared_error', max_depth=4, random_state = 10)
random.fit(x_treino, y_treino)

In [ ]:
random.score(x_treino, y_treino)

In [ ]:
random.score(x_teste, y_teste)

In [ ]:
previsoes_teste = random.predict(x_teste)
previsoes_teste

In [ ]:
mean_absolute_error(y_teste, previsoes_teste)

In [ ]:
mean_squared_error(y_teste, previsoes_teste)

In [ ]:
np.sqrt(mean_squared_error(y_teste, previsoes_teste))

In [ ]:
kfold = KFold(n_splits = 15, shuffle=True, random_state = 5)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
modelo = RandomForestRegressor(n_estimators=100, criterion='squared_error', max_depth=4, random_state = 10)
resultado = cross_val_score(modelo, previsores_esc, alvo, cv = kfold)
resultado

In [ ]:
print("Coeficiente de Determinação Médio: %.2f%%" % (resultado.mean() * 100.0))

### Regressão com XGBoost

In [ ]:
xgboost = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.05, objective="reg:squarederror", random_state=10)
xgboost.fit(x_treino, y_treino)

In [ ]:
xgboost.score(x_treino, y_treino)

In [ ]:
xgboost.score(x_teste, y_teste)

In [ ]:
previsoes_teste = xgboost.predict(x_teste)

In [ ]:
mean_absolute_error(y_teste, previsoes_teste)

In [ ]:
np.sqrt(mean_squared_error(y_teste, previsoes_teste))

In [ ]:
kfold = KFold(n_splits = 15, shuffle=True, random_state = 5)

In [ ]:
modelo = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.05, objective = "reg:squarederror")
resultado = cross_val_score(modelo, previsores_esc, alvo, cv = kfold)
resultado

In [ ]:
print("Coeficiente de determinação Médio: %.2f%%" % (resultado.mean() * 100.0))

### Regressão com LightBGM

In [ ]:
!pip install lightgbm

In [ ]:
lgbm = lgb.LGBMRegressor(num_leaves=50, max_depth=3, learning_rate=0.1, n_estimators=50, random_state=10)
lgbm.fit(x_treino, y_treino)

In [ ]:
lgbm.score(x_treino, y_treino)

In [ ]:
lgbm.score(x_teste, y_teste)

In [ ]:
previsoes_teste = lgbm.predict(x_teste)

In [ ]:
mean_absolute_error(y_teste, previsoes_teste)

In [ ]:
np.sqrt(mean_squared_error(y_teste, previsoes_teste))

In [ ]:
modelo = lgb.LGBMRegressor(num_leaves=50, max_depth=3, learning_rate=0.1, n_estimators=50)
resultado = cross_val_score(modelo, previsores_esc, alvo, cv = kfold)
resultado

In [ ]:
print("Coeficiente de Determinação Médio: %.2f%%" % (resultado.mean() * 100.0))

### Regressão com CatBoost

In [ ]:
catboost = CatBoostRegressor (iterations=80, learning_rate=0.08, depth = 4, random_state = 10)
catboost.fit(x_treino, y_treino)

In [ ]:
catboost.score(x_treino, y_treino)

In [ ]:
catboost.score(x_teste, y_teste)

In [ ]:
previsoes_teste = catboost.predict(x_teste)

In [ ]:
mean_absolute_error(y_teste, previsoes_teste)

In [ ]:
np.sqrt(mean_squared_error(y_teste, previsoes_teste))

In [ ]:
modelo = CatBoostRegressor (iterations=80, learning_rate=0.08, depth = 4, random_state = 10)
resultado = cross_val_score(modelo, previsores_esc, alvo, cv = kfold)
resultado

In [ ]:
print("Coeficiente de Determinação Médio: %.2f%%" % (resultado.mean() * 100.0))

### Redes Neurais

In [ ]:
redes = MLPRegressor(hidden_layer_sizes=(100, 100), activation='relu', verbose=True, max_iter=2000,
                    solver='adam', random_state = 12)

In [ ]:
redes.fit(x_treino, y_treino)

In [ ]:
redes.n_layers_

In [ ]:
redes.score(x_treino, y_treino)

In [ ]:
redes.score(x_teste, y_teste)

In [ ]:
previsoes_teste = redes.predict(x_teste)

In [ ]:
mean_absolute_error(y_teste, previsoes_teste)

In [ ]:
np.sqrt(mean_squared_error(y_teste, previsoes_teste))

In [ ]:
kfold = KFold(n_splits = 12, shuffle=True, random_state = 5)

In [ ]:
modelo = MLPRegressor(hidden_layer_sizes=(100, 100), activation='relu', verbose=True, max_iter=2000,
                    solver='adam', random_state = 12)
resultado = cross_val_score(modelo, previsores_esc, alvo, cv = kfold)
resultado

In [ ]:
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def avaliar_modelo(nome, modelo, x_treino, y_treino, x_teste, y_teste):
    # Treinamento
    modelo.fit(x_treino, y_treino)

    # Previsões
    y_pred_treino = modelo.predict(x_treino)
    y_pred_teste = modelo.predict(x_teste)

    # Métricas
    resultados = {
        "Modelo": nome,
        "R2_treino": r2_score(y_treino, y_pred_treino),
        "R2_teste": r2_score(y_teste, y_pred_teste),
        "MAE": mean_absolute_error(y_teste, y_pred_teste),
        "RMSE": np.sqrt(mean_squared_error(y_teste, y_pred_teste))
    }
    return resultados

# Lista de modelos para comparar
modelos = {
    "Regressão Linear": LinearRegression(),
    "SVR": SVR(kernel="rbf"),
    "Árvore de Decisão": DecisionTreeRegressor(max_depth=5, random_state=10),
    "Random Forest": RandomForestRegressor(n_estimators=60, max_depth=5, random_state=10),
    "XGBoost": XGBRegressor(n_estimators=180, max_depth=8, learning_rate=0.05, random_state=10),
    "LightGBM": lgb.LGBMRegressor(num_leaves=10, max_depth=10, learning_rate=0.1, n_estimators=60, random_state=10),
    "CatBoost": CatBoostRegressor(iterations=130, learning_rate=0.08, depth=5, random_state=10, verbose=False),
    "Rede Neural (MLP)": MLPRegressor(hidden_layer_sizes=(100,100), activation="relu", max_iter=2000, random_state=12)
}

# Avaliação de todos os modelos
resultados = []
for nome, modelo in modelos.items():
    resultados.append(avaliar_modelo(nome, modelo, x_treino, y_treino, x_teste, y_teste))

# Tabela comparativa
df_resultados = pd.DataFrame(resultados)
print(df_resultados.sort_values(by="R2_teste", ascending=False))


### Resultados:

**Regressão linear múltipla:** R^2 = 0,74/0,78; RMSE = 5796.28; R^2 Validação Cruzada: 73.54%

**Vetores de suporte:** R^2 = 0.85/0.86; RMSE = 4621.43; R^2 Validação Cruzada: 83.36%

**Árvore de decisão:** R^2 = 0.87/0,86; RMSE = 4592.76; R^2 Validação Cruzada: 83.83%

**Random Forest:** R^2 = 0,87/0,87; RMSE = 4466.85; R^2 Validação Cruzada: 85.26%

**XGBoost: R^2 = 0,88/0,88; RMSE = 4231.73; R^2 Validação Cruzada: 85.82%**

**LightBGM:** R^2 = 0,88/0,88; RMSE = 4317.04; R^2 Validação Cruzada: 85.68%

**CatBoost:** R^2 = 0,87/0,88; RMSE = 4271.98; R^2 Validação Cruzada: 85.54%

**Redes Neurais:** R^2 = 0,86/0,86; RMSE = 4699.10; R^2 Validação Cruzada: 83.18%